In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Flatten, Dense, Input
 
class LeNet:

    @staticmethod 
    def build(input_shape=(32,32,1), activation='sigmoid'):
        model = Sequential()
        model.add(Input(shape=input_shape))

        # 첫 번째 CONV => ACTIVATION => POOL 계층 
        model.add(Conv2D(6, 5, activation=activation,
                         kernel_initializer='random_uniform'))
        model.add(AveragePooling2D(pool_size=(2,2)))

        # 두 번째 CONV => ACTIVATION => POOL 계층 
        model.add(Conv2D(16, 5, 
                         activation=activation,
                         kernel_initializer='random_uniform'))
        model.add(AveragePooling2D(pool_size=(2,2)))


        # 첫 번째 FC 계층 
        model.add(Flatten())
        model.add(Dense(120, activation=activation))
        # 두 번째 FC 계층 
        model.add(Dense(84, activation=activation))
        # 출력층 soft-max 활성화 함수 사용
        model.add(Dense(10, activation='softmax'))

        return model

In [2]:
model = LeNet.build(input_shape=(28, 28, 1), activation="relu")
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 24, 24, 6)           │             156 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d (AveragePooling2D) │ (None, 12, 12, 6)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 8, 8, 16)            │           2,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_1                  │ (None, 4, 4, 16)            │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 120)                 │          30,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 84)                  │          10,164 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │             850 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 44,426 (173.54 KB)

 Trainable params: 44,426 (173.54 KB)

 Non-trainable params: 0 (0.00 B)

In [3]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd", 
              metrics=['accuracy'])

In [4]:
from tensorflow.keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 28, 28, 1) / 255.0
X_test = X_test.reshape(-1, 28, 28, 1) / 255.0

In [5]:
hist = model.fit(X_train, y_train,
                 batch_size=200, epochs=20)

Epoch 1/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.1499 - loss: 2.2981
Epoch 2/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.2942 - loss: 2.2239
Epoch 3/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7901 - loss: 0.7808
Epoch 4/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8756 - loss: 0.4145
Epoch 5/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9032 - loss: 0.3265
Epoch 6/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9154 - loss: 0.2854
Epoch 7/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9237 - loss: 0.2503
Epoch 8/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9289 - loss: 0.2312
Epoch 9/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9351 - loss: 0.2129
Epoch 10/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9372 - loss: 0.2051
Epoch 11/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9429 - loss: 0.1872
Epoch 12/20
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step

In [7]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 96.50%


In [8]:
import numpy as np
import cv2

samples = np.random.choice(np.arange(0,len(y_test)),size=(10,))

for i in samples:
    probs = model.predict(X_test[np.newaxis, i], verbose=0)
    prediction = probs.argmax(axis=1)
    image = (X_test[i] * 255).astype("uint8")
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    image = cv2.resize(image, (96, 96), 
                       interpolation=cv2.INTER_LINEAR)
 
    cv2.putText(image, str(prediction[0]), (5, 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 2)
    print("Predicted: {}, Actual: {}".format(prediction[0], y_test[i]))
    cv2.imshow("Digit", image)
    cv2.waitKey(0)

cv2.destroyAllWindows()

Predicted: 1, Actual: 1
Predicted: 7, Actual: 7
Predicted: 3, Actual: 3
Predicted: 6, Actual: 6
Predicted: 9, Actual: 4
Predicted: 8, Actual: 8
Predicted: 4, Actual: 4
Predicted: 3, Actual: 3
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0


In [9]:
model.save('mnist-lenet.keras')